In [22]:
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb

### 提取清洗完成的曝光

In [13]:
edf = pd.read_csv('J:/Code/Tencent_ad/2019/testA/log/del_rep_new/expose_days_bid_mean0513.csv')
edf
op = pd.read_csv('J:/Code/Tencent_ad/2019/testA/op/build_op.csv')
op.head()
aid_use = op['aid'].value_counts().index.values
aid_use
op.head()
op0 = pd.read_csv('J:/Code/Tencent_ad/2019/testA/op/ad_operation_0.csv')


### 针对static和operation构建特征

In [2]:
st = pd.read_csv('J:/Code/Tencent_ad/2019/testA/static/ad_static_feature_re_time.csv')
op_time = pd.read_csv('J:/Code/Tencent_ad/2019/testA/op/ad_operation_advertising_duration.csv')

st_new = pd.DataFrame()
st_new = st[['aid', 'material_size', 'ad_account_id', 'ad_class_id', 'goods_class', 'goods_id', 'build_time_category']]
st_new.head()
st_new['old'] = 1
st_new.head()

gdf_tmp = st_new.groupby('ad_account_id').size()
gdf_tmp = gdf_tmp.reset_index()
gdf_tmp.columns = ['ad_account_id', 'ad_nums']
gdf_tmp
st_new = pd.merge(st_new, gdf_tmp, on = 'ad_account_id', how = 'left')
st_new

st_new['ad_class_id'] = st_new['ad_class_id'].apply(str)

ad_class_len = []
ad_class_id = []
for i,line in tqdm(enumerate(st_new['ad_class_id'])):
    line = line.strip().split(',')
    ad_class_len.append(len(line))
    ad_class_id.append(line[0])
st_new['ad_class_len'] = ad_class_len
st_new['ad_class_id'] = ad_class_id

st_new['goods_id'] = st_new['goods_id'].apply(str)
goods_id_len = []
goods_id = []
for i,line in tqdm(enumerate(st_new['goods_id'])):
    line = line.strip().split(',')
    goods_id_len.append(len(line))
    goods_id.append(line[0])
st_new['goods_id_len'] = goods_id_len
st_new['goods_id'] = goods_id

st_new[st_new['ad_class_len'] > 1]
re_index = st_new[st_new['build_time_category'] == 197011].index.values
st_new['build_time_category'].loc[re_index] = 2019315
for feature in st_new.columns:
    st_new[feature].astype(int)

st_new

C:\Users\yyhhlancelot\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\yyhhlancelot\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
509251it [00:00, 1244839.87it/s]
509251it [00:00, 1238679.84it/s]
C:\Users\yyhhlancelot\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, 

,aid,material_size,ad_account_id,ad_class_id,goods_class,goods_id,build_time_category,old,ad_nums,ad_class_len,goods_id_len
0,233649,1,25681,136,13,7356,2018929,1,4,1,1
1,547531,40,20696,186,1,-1,2019221,1,4,1,1
2,707841,40,3968,186,3,-1,201936,1,135,1,1
3,733436,64,22405,117,5,31722,2019319,1,190,1,1
4,249105,44,11360,145,18,29999,2019315,1,3066,1,1
5,160014,36,6441,198,18,2373,2019314,1,151,1,1
6,541096,44,5117,232,5,220,2019313,1,578,1,1
7,634000,64,28588,76,18,20164,2019311,1,1603,1,1
8,367826,64,26093,221,1,-1,201939,1,79,1,1
9,660913,64,10035,164,3,24783,201937,1,105,1,1


### 构建训练集

In [14]:
train_df = pd.DataFrame()
train_df['aid'] = aid_use
train_df = pd.merge(train_df, st_new, on = ['aid'], how = 'left')
train_df = pd.merge(train_df, edf, on = ['aid'], how = 'left')
# train_df = pd.merge(train_df, op_time[['aid', 'time_mean']], on = 'aid', how = 'left')
tmp_list = []
train_df['ad_class_id'] = train_df['ad_class_id'].apply(str)
for i, line in tqdm(enumerate(train_df['ad_class_id'])):
    if len(line) > 3:
        tmp_list.append(i)
for i in tqdm(tmp_list):
    line = train_df['ad_class_id'][i].strip().split(',')
    train_df['ad_class_id'][i] = line[0]


def one_num(tmp_str):
    len_ = len(tmp_str)
    num = 0
    for j in range(len_):
        if tmp_str[j] == '1':
            num = num + 1
    return num

null_time_aid = train_df['aid']
tmp_time = []
for i in tqdm(null_time_aid):
    tmp_gdf = op0[op0['aid'] == i]
    tmp_gdf = tmp_gdf.reset_index()
    line = tmp_gdf['modify_val'][2]
    line = line.strip().split(',')
    for i, val in enumerate(line):
        if val != '0':
            num = one_num(str(bin(int(val))))
            break
    tmp_time.append(num/2)

train_df['time_mean'] = tmp_time # 使用的是st里面的第一个非零时间


for feature in train_df.columns:
    print(feature)
    train_df[feature] = train_df[feature].astype(int)
    
    
train_df.head()

175351it [00:00, 2331837.06it/s]
0it [00:00, ?it/s]
100%|█████████████████████████████████████████████████████████████████████████| 175351/175351 [05:05<00:00, 573.52it/s]


aid
material_size
ad_account_id
ad_class_id
goods_class
goods_id
build_time_category
old
ad_nums
ad_class_len
goods_id_len
year
month
day
expose_days
bid_mean
time_mean


,aid,material_size,ad_account_id,ad_class_id,goods_class,goods_id,build_time_category,old,ad_nums,ad_class_len,goods_id_len,year,month,day,expose_days,bid_mean,time_mean
0,363038,40,6096,37,1,-1,2019317,1,2,1,1,2019,3,18,63,97,16
1,363038,40,6096,37,1,-1,2019317,1,2,1,1,2019,3,19,31,107,16
2,99102,30,7955,230,1,-1,2019125,1,50,1,1,2019,2,20,12,102,11
3,99102,30,7955,230,1,-1,2019125,1,50,1,1,2019,2,24,7,102,11
4,99102,30,7955,230,1,-1,2019125,1,50,1,1,2019,3,1,19,138,11


### 构建测试集特征

In [15]:


test_ori2 = pd.read_csv('J:/Code/Tencent_ad/2019/testA/test/testB_feature_0.csv')
test_ori2['build_time'] = pd.to_datetime(test_ori2['build_time'].values, utc=True, unit='s').tz_convert(
            "Asia/Shanghai")
test_ori2['year'] = test_ori2['build_time'].dt.year
test_ori2['month'] = test_ori2['build_time'].dt.month
test_ori2['day'] = test_ori2['build_time'].dt.day
build_time_category = []
len_ = test_ori2.shape[0]
for i in tqdm(range(len_)):
    build_time_category.append(str(test_ori2['year'][i]) + str(test_ori2['month'][i]) + str(test_ori2['day'][i]))

test_ori2['build_time_category'] = build_time_category
test_df2 = pd.DataFrame()
test_df2 = test_ori2[['aid', 'bid', 'material_size', 'ad_account_id', 'ad_class_id', 'goods_class', 'goods_id', 'build_time_category']]
test_df2 = test_df2.astype(int)
test_df2 = pd.merge(test_df2, st_new[['aid', 'old', 'ad_nums', 'ad_class_len', 'goods_id_len']], on = ['aid'], how = 'left')


test_df2['old'].fillna(0, inplace = True)
test_df2['ad_class_len'].fillna(1, inplace = True)
test_df2['goods_id_len'].fillna(1, inplace = True)

null_list = test_df2[test_df2['ad_nums'].isnull()].index.values
ad_nums = [] 
for i in null_list:
    tmp_df = test_df2[test_df2['ad_account_id'] == test_df2['ad_account_id'][i]]
    ad_nums.append(tmp_df['aid'].value_counts().shape[0])
test_df2['ad_nums'].loc[null_list] = ad_nums

tmp_time_df = train_df[['aid', 'time_mean']]
tmp_time_df2 = tmp_time_df.drop_duplicates()
test_df2 = pd.merge(test_df2, tmp_time_df2, on = 'aid', how = 'left')
test_df2['time_mean'].value_counts()


test_df2['time_mean'].fillna(24, inplace = True)
test_df2

tmp_gdf = test_df2.groupby(['aid']).agg({'bid' : 'mean'})
tmp_gdf = tmp_gdf.reset_index()
tmp_gdf.columns = ['aid', 'bid_mean']
test_df2 = pd.merge(test_df2, tmp_gdf, on = 'aid', how = 'left')
test_df2

100%|█████████████████████████████████████████████████████████████████████████| 38596/38596 [00:01<00:00, 24722.78it/s]


,aid,bid,material_size,ad_account_id,ad_class_id,goods_class,goods_id,build_time_category,old,ad_nums,ad_class_len,goods_id_len,time_mean,bid_mean
0,152978,37,30,23511,118,3,-1,20161129,1.0,25.0,1.0,1.0,24.0,27.900000
1,507977,54,30,4211,122,13,629,201935,1.0,2.0,1.0,1.0,17.0,35.300000
2,639943,161,34,15582,84,13,2525,2018621,1.0,4.0,1.0,1.0,17.0,167.545455
3,605859,28,40,10907,98,13,3808,2019321,0.0,6.0,1.0,1.0,24.0,28.800000
4,742582,12,40,15855,122,13,7017,2019228,0.0,6.0,1.0,1.0,24.0,104.454545
5,742243,34,30,19328,146,13,7904,2018123,0.0,4.0,1.0,1.0,24.0,63.500000
6,306151,1,40,6262,221,1,-1,2019321,0.0,84.0,1.0,1.0,24.0,23.888889
7,739668,10,64,2347,54,15,2396,201935,0.0,3.0,1.0,1.0,24.0,38.545455
8,443852,66,30,6678,122,13,4332,201936,1.0,3.0,1.0,1.0,17.0,81.818182
9,647165,1827,1,2930,33,1,-1,2019320,0.0,1.0,1.0,1.0,24.0,1226.909091


In [18]:
re_index = train_df[train_df['ad_class_id'] < 0].index.values
train_df['ad_class_id'].loc[re_index] = 122
train_df[train_df['ad_class_id'] < 0]
# test_df['ad_class_id'].value_counts()
re_index = test_df2[test_df2['ad_class_id'] < 0].index.values
test_df2['ad_class_id'].loc[re_index] = 122
test_df2[test_df2['ad_class_id'] < 0]

,aid,bid,material_size,ad_account_id,ad_class_id,goods_class,goods_id,build_time_category,old,ad_nums,ad_class_len,goods_id_len,time_mean,bid_mean


### 模型训练

In [19]:
from sklearn.preprocessing import OneHotEncoder
from scipy import sparse
## onehot encoder
numerical_feature = ['material_size', 'old', 'ad_nums', 'ad_class_len', 'goods_id_len',
                    'bid_mean', 'time_mean']
one_hot_feature = ['ad_account_id', 'ad_class_id', 'goods_class',
        'build_time_category'] #'goods_id'
enc = OneHotEncoder()
train_x = train_df[numerical_feature]
test_x = test_df2[numerical_feature]
train_y = train_df['expose_days']

train_X = train_df[one_hot_feature]
test_X = train_df[one_hot_feature]

data = pd.concat([train_X,test_X])

for feature in one_hot_feature:
    print(feature)
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train_df[feature].values.reshape(-1, 1))
    test_a = enc.transform(test_df2[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a))
    test_x = sparse.hstack((test_x, test_a)) 

In [23]:
gbm = lgb.LGBMRegressor(objective='regression_l1',
                        num_leaves=50,
                        learning_rate=0.15,
                        n_estimators=3000)#2500 86分

gbm

gbm.fit(train_x, train_y, eval_metric='mae')

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.15, max_depth=-1,
       min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
       n_estimators=3000, n_jobs=-1, num_leaves=50,
       objective='regression_l1', random_state=None, reg_alpha=0.0,
       reg_lambda=0.0, silent=True, subsample=1.0,
       subsample_for_bin=200000, subsample_freq=0)

In [24]:
pred_b = gbm.predict(test_x)
pred_b

C:\Users\yyhhlancelot\Anaconda3\lib\site-packages\lightgbm\basic.py:452: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


array([  9.81609024,  18.51361904,   1.75675041, ...,   0.97708345,
        11.65713708,  66.65473513])

In [32]:
test_df2.drop(['expose_days_mean'], axis = 1, inplace = True)

### 使用规则模型

In [33]:
gdf_tmp = edf[['aid', 'expose_days']].groupby('aid').agg({'expose_days' : 'mean'})
gdf_tmp = gdf_tmp.reset_index()
gdf_tmp.columns = ['aid', 'expose_days_mean']

test_df2 = pd.merge(test_df2, gdf_tmp, on = 'aid', how = 'left')
test_df2

,aid,bid,material_size,ad_account_id,ad_class_id,goods_class,goods_id,build_time_category,old,ad_nums,ad_class_len,goods_id_len,time_mean,bid_mean,expose_days_mean
0,152978,37,30,23511,118,3,-1,20161129,1.0,25.0,1.0,1.0,24.0,27.900000,12.529412
1,507977,54,30,4211,122,13,629,201935,1.0,2.0,1.0,1.0,17.0,35.300000,24.200000
2,639943,161,34,15582,84,13,2525,2018621,1.0,4.0,1.0,1.0,17.0,167.545455,2.875000
3,605859,28,40,10907,98,13,3808,2019321,0.0,6.0,1.0,1.0,24.0,28.800000,NaN
4,742582,12,40,15855,122,13,7017,2019228,0.0,6.0,1.0,1.0,24.0,104.454545,NaN
5,742243,34,30,19328,146,13,7904,2018123,0.0,4.0,1.0,1.0,24.0,63.500000,NaN
6,306151,1,40,6262,221,1,-1,2019321,0.0,84.0,1.0,1.0,24.0,23.888889,NaN
7,739668,10,64,2347,54,15,2396,201935,0.0,3.0,1.0,1.0,24.0,38.545455,NaN
8,443852,66,30,6678,122,13,4332,201936,1.0,3.0,1.0,1.0,17.0,81.818182,12.153846
9,647165,1827,1,2930,33,1,-1,2019320,0.0,1.0,1.0,1.0,24.0,1226.909091,NaN


In [34]:
re_index = test_df2[test_df2['expose_days_mean'].isnull()].index.values
test_df2['expose_days_mean'].loc[re_index] = pred_b[re_index]
test_df2

,aid,bid,material_size,ad_account_id,ad_class_id,goods_class,goods_id,build_time_category,old,ad_nums,ad_class_len,goods_id_len,time_mean,bid_mean,expose_days_mean
0,152978,37,30,23511,118,3,-1,20161129,1.0,25.0,1.0,1.0,24.0,27.900000,12.529412
1,507977,54,30,4211,122,13,629,201935,1.0,2.0,1.0,1.0,17.0,35.300000,24.200000
2,639943,161,34,15582,84,13,2525,2018621,1.0,4.0,1.0,1.0,17.0,167.545455,2.875000
3,605859,28,40,10907,98,13,3808,2019321,0.0,6.0,1.0,1.0,24.0,28.800000,10.868211
4,742582,12,40,15855,122,13,7017,2019228,0.0,6.0,1.0,1.0,24.0,104.454545,13.059412
5,742243,34,30,19328,146,13,7904,2018123,0.0,4.0,1.0,1.0,24.0,63.500000,5.100751
6,306151,1,40,6262,221,1,-1,2019321,0.0,84.0,1.0,1.0,24.0,23.888889,40.420127
7,739668,10,64,2347,54,15,2396,201935,0.0,3.0,1.0,1.0,24.0,38.545455,9.369342
8,443852,66,30,6678,122,13,4332,201936,1.0,3.0,1.0,1.0,17.0,81.818182,12.153846
9,647165,1827,1,2930,33,1,-1,2019320,0.0,1.0,1.0,1.0,24.0,1226.909091,2.741764


### 调整单调性

In [39]:
test_df2['expose_days_mean'].loc[test_df2[test_df2['expose_days_mean'] < 0].index.values] = 1
test_df2['final'] = test_df2['expose_days_mean'] + test_df2['bid'] * 0.001
test_df2

,aid,bid,material_size,ad_account_id,ad_class_id,goods_class,goods_id,build_time_category,old,ad_nums,ad_class_len,goods_id_len,time_mean,bid_mean,expose_days_mean,final
0,152978,37,30,23511,118,3,-1,20161129,1.0,25.0,1.0,1.0,24.0,27.900000,12.529412,12.566412
1,507977,54,30,4211,122,13,629,201935,1.0,2.0,1.0,1.0,17.0,35.300000,24.200000,24.254000
2,639943,161,34,15582,84,13,2525,2018621,1.0,4.0,1.0,1.0,17.0,167.545455,2.875000,3.036000
3,605859,28,40,10907,98,13,3808,2019321,0.0,6.0,1.0,1.0,24.0,28.800000,10.868211,10.896211
4,742582,12,40,15855,122,13,7017,2019228,0.0,6.0,1.0,1.0,24.0,104.454545,13.059412,13.071412
5,742243,34,30,19328,146,13,7904,2018123,0.0,4.0,1.0,1.0,24.0,63.500000,5.100751,5.134751
6,306151,1,40,6262,221,1,-1,2019321,0.0,84.0,1.0,1.0,24.0,23.888889,40.420127,40.421127
7,739668,10,64,2347,54,15,2396,201935,0.0,3.0,1.0,1.0,24.0,38.545455,9.369342,9.379342
8,443852,66,30,6678,122,13,4332,201936,1.0,3.0,1.0,1.0,17.0,81.818182,12.153846,12.219846
9,647165,1827,1,2930,33,1,-1,2019320,0.0,1.0,1.0,1.0,24.0,1226.909091,2.741764,4.568764


In [42]:
subB = pd.DataFrame()
subB['id'] = test_ori2['id']
subB['pred_final'] = test_df2['final'].apply(lambda x : round(x, 4))
subB.to_csv('J:/Code/Tencent_ad/2019/testA/submission/submission.csv', index = False, header = False)